In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using ForwardDiff
using HybridRobotDynamics
using HybridTrajIpopt
using ImpactSafety

In [ ]:
"""
Set up system and controller
"""

sys = bouncing_ball()

# Define stage and terminal cost functions
Q = 1e0 * diagm([1.0, 1.0, 0.0, 0.0])
R = Matrix(1e-6 * I(sys.nu))
Qf = 1e3 * Q
stage = (x, u) -> x'*Q*x + u'*R*u
terminal = x -> x'*Qf*x

# Define trajopt parameters
N = 50
Δt = 0.01
hs = ImplicitIntegrator(:hermite_simpson)
params = ProblemParameters(sys, hs, stage, terminal, N; Δt)

# Define transition sequence and terminal guard
impact = sys.transitions[:impact]
sequence = [
    TransitionTiming(10, impact),
    TransitionTiming(20, impact),
    TransitionTiming(30, impact),
    TransitionTiming(40, impact),
]
term_guard = impact.guard

# Define reference trajectory and initial conditions
xic = [0.0; 10.0; 10.0; 0.0]
xgc = [5.0; 5.0; 0.0; 0.0]
xrefs = repeat(xgc, N)
urefs = zeros((N-1) * sys.nu)

# Define solver callbacks
cb = SolverCallbacks(
    params, sequence, term_guard, xrefs, urefs, xic;
    gauss_newton=true
)

nothing

In [ ]:
"""
Get TVLQR Policy
"""

# Solve using Ipopt
y0 = zeros(params.dims.ny)
sol = ipopt_solve(params, cb, y0)
xs, us = decompose_trajectory(params.idx, sol.x)
plot_2d_states(N, sys.nx, (1,2), xs)

# Init TVLQR policy with RK4 integration
yref = sol.x
rk4 = ExplicitIntegrator(:rk4)
tvlqr = TimeVaryingLQR(params, rk4, Q, R, Qf, sequence, yref)

nothing

In [ ]:
function post_impact_velocity(
    flow::Union{ControlAffineFlow, Function},
    trn::Transition,
    vidx::UnitRange{Int},
    x::Vector,
    u::Vector,
    Δt::Vector
)::Vector
    x1 = x + Δt*flow(x, u)
    xJ = trn.reset(x1)
    return xJ[vidx]
end

In [ ]:
"""
Obstacle avoidance auxiliary functions
"""

function opos(o::Vector)::Vector
    return o[1:2]
end

function ovel(o::Vector)::Vector
    return o[3:4]
end

function xpos(x::Vector)::Vector
    return x[1:2]
end

function xvel(x::Vector)::Vector
    return x[3:4]
end

function Φ(
    n::Real,
    k::Real,
    p::Real,
    q::Real,
    r::Real,
    dmin::Real,
    flow::ControlAffineFlow,
    trn::Transition,
    o::Vector,
    u::Vector,
    x::Vector
)::Vector
    rel_dist = opos(o) - xpos(x)
    d = norm(rel_dist)
    d_unit = rel_dist / d

    rel_vel = ovel(o) - xvel(x)
    v = sum(rel_vel .* d_unit)

    rel_vel_J = ovel(o) - xvel(trn.reset(x))
    vJ = sum(rel_vel_J .* d_unit)
    g = trn.guard(x)
    ġ = ForwardDiff.gradient(trn.guard, x)' * flow(x, u)
    println(p*vJ*exp(-q*ġ - r*g))
    return [dmin^n - d^n - k*v - p*vJ*exp(-q*ġ - r*g)]
end

function Φ̇ub(
    Φargs::Tuple,
    η::Real,
    x::Vector
)::Vector
    Φval = Φ(Φargs..., x)
    if Φval[1] > 1e-6
        return [-η]
    elseif Φval[1] > -1e-6
        return [0.0]
    else
        return [Inf]
    end
end

nothing

In [ ]:
"""
Set up safety index and safe set bound
"""

# Safety index params
n = 2
k = 0.5
p = 0.5
q = 0.5
r = 2.0
dmin = 1.5

# SSA params
η = 1

# Safety filter
no = 4
ϵ = 0.0
ssa = SafetyFilter(sys.nu, ϵ, Φ, Φ̇ub, sys.modes[:flight].flow)

# Obstacle state
o = [4.0, 4.0, 0.0, 0.0]

nothing


In [ ]:
"""
Simulate the combined policy
"""

# Sim settings
speedup = 100
Δtsim = Δt / speedup
Nsim = 1 + (N-1)*speedup

# Timing mapping from sim to TVLQR
kmap = length(tvlqr.idx.u) / Nsim

# Initial conditions
xs_sim = [zeros(sys.nx) for k = 1:Nsim]
us_sim = [zeros(sys.nu) for k = 1:(Nsim-1)]
xs_sim[1] = xic
mI = sys.modes[:flight]

# Simulate
for k = 1 : (Nsim-1)
    # Nominal controller
    unom = tvlqr(xs_sim[k], trunc(Int, 1 + k*kmap))

    # Filtered controller
    Φargs = (n, k, p, q, r, dmin, mI.flow, first(keys(mI.transitions)), o, unom)
    Φ̇args = (Φargs, η)
    us_sim[k] = ssa(Φargs, Φ̇args, xs_sim[k], unom)

    # Integrate smooth dynamics
    xs_sim[k+1] = rk4(mI.flow, xs_sim[k], us_sim[k], Δtsim)

    # Reset and update mode if guard is hit
    for (trans, mJ) in mI.transitions
        if trans.guard(xs_sim[k+1]) < 0.0
            xs_sim[k+1] = trans.reset(xs_sim[k+1])
            mI = mJ
            break
        end
    end
    #@show Φanon(o, xs_sim[k+1])
end

# Visualize
plot_2d_states(Nsim, sys.nx, (1,2), vcat(xs_sim...))

nothing